### 22H1120016 - Trần Đăng Nam 

Sử dụng dataset https://archive.ics.uci.edu/ml/datasets/FMA%3A+A+Dataset+For+Music+Analysis

(hoặc download ở đây: https://github.com/mdeff/fma)

đề xuất giải pháp và cài đặt ứng dụng gom cụm các bản nhạc Mp3.


Giải pháp: do  dữ liệu  khá lớn ->  nên dùng  K-Means, vì mô  hình này xử  lý  nhanh  hơn  cho dữ liệu lớn.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [ ]:
# Bước 1: Đọc dữ liệu từ file CSV
# Giả sử bạn đã tải và giải nén fma_metadata.zip
tracks = pd.read_csv('tracks.csv', header=[0, 1], index_col=0)  # Metadata
features = pd.read_csv('features.csv', header=[0, 1, 2], index_col=0)  # Đặc trưng âm thanh

# Lọc dữ liệu thuộc tập "small" (fma_small)
small_tracks = tracks[tracks['set', 'subset'] <= 'small']
small_features = features.loc[small_tracks.index]


In [ ]:
# Bước 2: Làm sạch và chuẩn bị dữ liệu
# Loại bỏ các hàng có giá trị NaN
small_features = small_features.dropna()
small_tracks = small_tracks.loc[small_features.index]

# Lấy tất cả đặc trưng âm thanh (gộp các cột từ các nhóm như chroma, mfcc, spectral, v.v.)
X = small_features.values

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:

# Bước 3: Gom cụm với K-Means
# Chọn k = 8 (vì fma_small có 8 thể loại cân bằng, ta thử gom thành 8 cụm)
k = 8
kmeans = KMeans(n_clusters=k, random_state=42)
labels = kmeans.fit_predict(X_scaled)

# Bước 4: Tạo DataFrame chứa kết quả gom cụm
clustering_results = pd.DataFrame({
    'Track_ID': small_tracks.index,
    'Title': small_tracks['track', 'title'],
    'Artist': small_tracks['artist', 'name'],
    'Cluster': labels
})



In [ ]:
# Lưu kết quả vào file CSV
clustering_results.to_csv('fma_clustering_results.csv', index=False)
print("Kết quả gom cụm đã được lưu vào file 'fma_clustering_results.csv'.")

# Bước 5: In một số thông tin để kiểm tra
print("\nSố lượng bản nhạc trong mỗi cụm:")
print(clustering_results['Cluster'].value_counts())

# In một số ví dụ từ mỗi cụm
print("\nVí dụ một số bản nhạc trong mỗi cụm:")
for cluster in range(k):
    print(f"\nCụm {cluster}:")
    cluster_tracks = clustering_results[clustering_results['Cluster'] == cluster].head(3)
    for _, row in cluster_tracks.iterrows():
        print(f"Track ID: {row['Track_ID']}, Title: {row['Title']}, Artist: {row['Artist']}")



In [ ]:
# (Tùy chọn) Vẽ biểu đồ phân bố cụm
plt.figure(figsize=(8, 6))
plt.hist(labels, bins=range(k+1), align='left', rwidth=0.8)
plt.title('Phân bố số lượng bản nhạc trong mỗi cụm')
plt.xlabel('Cụm')
plt.ylabel('Số lượng bản nhạc')
plt.xticks(range(k))
plt.grid(True)
plt.savefig('fma_cluster_distribution.png')